# Ensemble learning for image classification (Flower Recognition)

### Summary

Test

Group number and member names:

In [ ]:
GROUP = "36"
NAME1 = "Timothy Hellberg"
NAME2 = "Lars Liberg"

## 0. Imports

In [ ]:
# YOUR CODE HERE
# For dealing with files
import os, sys
import shutil

# For using regex expressions
import re

# For splitting the data
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

# Packages for defining the architecture of our model
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Input, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import regularizers
from keras.optimizers import Adam
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model 
from keras.wrappers.scikit_learn import KerasClassifier

# For generating data
from keras.preprocessing.image import ImageDataGenerator

# One-hot encoding
from keras.utils import np_utils

# Callbacks for training
from keras.callbacks import TensorBoard, EarlyStopping

# Ploting
import matplotlib.pyplot as plt
%matplotlib inline

# Ndarray computations
import numpy as np

# Confusion matrix for assessment step
from sklearn.metrics import confusion_matrix

# Other
import pandas as pd
from subprocess import check_output
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
from time import time
from time import sleep
from scipy import ndimage
from scipy.misc import imresize
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout, Maximum
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Concatenate
from keras import backend as K 

## 1. Loading the data and preprocessing

In [ ]:
#home_path = "/Users/timhell/Google Drive/11 Färdigheter/Programmering/Python/Deep machine learning/deep-machine-learning/Project"
home_path = "/home/student/deep-machine-learning/Project"
os.chdir(home_path)

image_height = 64
image_width = image_height
channels = 3

class_names = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
train_dir = home_path + '/train/'
test_dir = home_path + '/test/'
train_filenames = {}
test_filenames = {}
for i in range(5):
    os.chdir(train_dir + class_names[i])
    train_filenames_tmp = np.array(os.listdir())
    newlist = []
    for names in train_filenames_tmp:
        if names.endswith(".jpg"):
            newlist.append(names)
    train_filenames[i] = newlist
for i in range(5):
    os.chdir(test_dir + class_names[i])
    test_filenames_tmp = np.array(os.listdir())
    newlist = []
    for names in test_filenames_tmp:
        if names.endswith(".jpg"):
            newlist.append(names)
    test_filenames[i] = newlist
os.chdir(home_path)

n_train = len(train_filenames[0]) + len(train_filenames[1]) + len(train_filenames[2]) + len(train_filenames[3]) + len(train_filenames[4])
n_test = len(test_filenames[0]) + len(test_filenames[1]) + len(test_filenames[2]) + len(test_filenames[3]) + len(test_filenames[4])

print('n_train:', n_train)
print('n_test:', n_test)

x_train = np.zeros((n_train, image_width, image_height, channels), dtype=np.float32)
y_train = np.zeros((n_train, 1), dtype=int)
x_test = np.zeros((n_test, image_width, image_height, channels), dtype=np.float32)
y_test = np.zeros((n_test, 1), dtype=int)

index = 0
for class_i in range(5):
    #print('class_i:', class_i)
    for file_i in range(len(train_filenames[class_i])):
        #print('file_i_i:', file_i)
        img = load_img(train_dir + class_names[class_i] + '/' + train_filenames[class_i][file_i]) 
        x = img_to_array(img) 
        x = imresize(x, size=(image_height,image_width))
        x_train[index] = x[:,:,:]
        y_train[index] = class_i
        index += 1
        del x

index = 0
for class_i in range(5):
    #print('class_i:', class_i)
    for file_i in range(len(test_filenames[class_i])):
        #print('file_i_i:', file_i)
        img = load_img(test_dir + class_names[class_i] + '/' + test_filenames[class_i][file_i]) 
        x = img_to_array(img) 
        x = imresize(x, size=(image_height,image_width))
        x_test[index] = x[:,:,:]
        y_test[index] = class_i
        index += 1
        del x

x_train = x_train / 255.
x_test = x_test / 255.

indices = np.arange(n_train)
np.random.shuffle(indices)
x_train = x_train[indices,:,:,:]
y_train = y_train[indices]
indices = np.arange(n_test)
np.random.shuffle(indices)
x_test = x_test[indices,:,:,:]
y_test = y_test[indices]

y_train = to_categorical(y_train, num_classes=len(class_names))
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape,                                                                
                                                                                            x_test.shape, y_test.shape))
input_shape = x_train[0,:,:,:].shape
print("input_shape:",input_shape)
model_input = Input(shape=input_shape)

In [ ]:
i = np.random.randint(n_train)
print(np.argmax(y_train[i]))
#print(x_train[i])
print(class_names[int(np.argmax(y_train[i]))]);
plt.imshow(x_train[i]);

## 2. Training

### 2.1 Functions, learners and combiners

In [ ]:
# Universal functions

b_size = 128

def compile_and_train(model, num_epochs): 
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = home_path + '/weights/FR/' + model.name + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=y_train, batch_size=b_size, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    return history

def compile_and_train_bagging(model, num_epochs, x_train, y_train): 
    indices = np.random.randint(n_train, size=n_train)
    x_train_bag = x_train[indices,:,:,:]
    y_train_bag = y_train[indices,:]
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = home_path + '/weights/FR/' + model.name + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train_bag, y=y_train_bag, batch_size=b_size, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    del x_train_bag, y_train_bag
    return history

def evaluate_test_accuracy(model):
    pred = model.predict(x_test, batch_size = b_size)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    test_accuracy = np.sum(np.equal(pred, y_test)) / y_test.shape[0]  
    return test_accuracy

In [ ]:
# Custom base learners

def base_cnn_reg(model_input):
    x = Conv2D(128, (5, 5), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(model_input, x, name='FR_base_cnn_reg')
    return model

def base_cnn_2_reg(model_input): # same as the bagged model
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(model_input, x, name='FR_base_cnn_2_reg')
    return model

def base_cnn_2_reg_fewer_filters_and_dense_neurons(model_input):
    x = Conv2D(64, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(model_input, x, name='FR_base_cnn_2_reg_fewer_filters_and_dense_neurons')
    return model

def base_cnn_3_reg(model_input):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(model_input, x, name='FR_base_cnn_3_reg')
    return model

def base_cnn_4_conv_concise(model_input):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(model_input, x, name='FR_base_cnn_4_conv_concise')
    return model

def base_learner_bagging(model_input, num):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(model_input, x, name='FR_base_learner_' + str(num) + '_bagging')
    return model

In [ ]:
# Custom combiners

def average_ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    #print(np.shape(np.array(outputs)))
    #print(outputs[0])
    y = Average()(outputs)
    model = Model(models[0].input, y, name='FR_average_ensemble')
    return model

def concatenate_ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    print(np.shape(np.array(outputs)))
    y = Concatenate(axis=-1)(outputs)
    model = Model(model_input, y, name='FR_concatenate_ensemble')
    return model

def basic_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(5, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='FR_basic_stacking_' + str(num) + '_ensemble')
    return model

def small_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(5, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='FR_small_stacking_' + str(num) + '_ensemble')
    return model

def large_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(5, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='FR_large_stacking_' + str(num) + '_ensemble')
    return model

def deep_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(5, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='FR_deep_stacking_' + str(num) + '_ensemble')
    return model

def bn_deep_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(5, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='FR_bn_deep_stacking_' + str(num) + '_ensemble')
    return model

### 2.2 Train base learners

#### 2.2.1 Bagging (train base learners)

In [ ]:
# Train bagged model
base_model = base_learner_bagging(model_input, 3)
print(base_model.summary())
del base_model

models = []
for i in np.arange(0,50):
    base_model = base_learner_bagging(model_input, i)
    _ = compile_and_train_bagging(base_model, 20, x_train, y_train)
    models.append(base_model)
    base_model.save_weights('weights_FR/weights_FR_base_learner_' + str(i) + '_bagging.hdf5')
    del base_model


In [ ]:
# Train single model with full dataset
base_model_full = base_learner_bagging(model_input, 50)
_ = compile_and_train(base_model_full, 30)
#_ = compile_and_train_bagging(base_model_full, 5, x_train, y_train)
print(evaluate_test_accuracy(base_model_full))

In [ ]:
# Load bagged model (only weights)

models_bag = []
for i in np.arange(0,50):
    base_model = base_learner_bagging(model_input, i)
    base_model.load_weights('weights_FR/weights_FR_base_learner_' + str(i) + '_bagging.hdf5')
    models_bag.append(base_model)
    print(i)

In [ ]:
# Freeze weights in bagged model

model_0 = models_bag[2]
model_0.summary()

for model in models_bag:
    for layer in model.layers:
        layer.trainable = False
        
model_0.summary()

In [ ]:
# Evaluate the different bagged models

for model in models_bag:
    print(evaluate_test_accuracy(model))


#### 2.2.2 Non-bagging (train base learners)

In [ ]:
# Train models

base_model = base_cnn_reg(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_2_reg(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_2_reg_fewer_filters_and_dense_neurons(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_3_reg(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_4_conv_concise(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')

In [ ]:
# Train single model

base_model = base_cnn_reg(model_input)
_ = compile_and_train(base_model, 10)
base_model.save_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')

In [ ]:
# Train model without saving weights

base_model = base_cnn(model_input)
print(base_model.summary())

_ = compile_and_train(base_model, 10)

In [ ]:
# Load models (only weights)

models_nonbag = []

base_model = base_cnn_reg(model_input)
base_model.load_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_2_reg(model_input)
base_model.load_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_2_reg_fewer_filters_and_dense_neurons(model_input)
base_model.load_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_3_reg(model_input)
base_model.load_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_4_conv_concise(model_input)
base_model.load_weights('weights_FR/weights_FR_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)


In [ ]:
# Freeze weights in non-bagged model

model_0 = models_nonbag[0]
model_0.summary()

for model in models_nonbag:
    for layer in model.layers:
        layer.trainable = False
        
model_0.summary()

In [ ]:
# Evaluate the different non-bagged models

for model in models_nonbag:
    print(evaluate_test_accuracy(model))

### 2.3 Evaluate simple combiners

#### 2.3.1 Bagging (evaluate simple combiners)

In [ ]:
# Max voting

max_voting_accuracies = np.zeros((51,1))
for k in np.arange(2,50):
    n_models = k
    models_short = models_bag[0:n_models]

    pred_new = np.zeros((n_test,5))
    max_voting_model = concatenate_ensemble(models_short, model_input)
    pred = max_voting_model.predict(x_test, batch_size = 128)
    pred = np.expand_dims(pred, axis=1)
    for i in range(n_models):
        j = i*5
        #print(i)
        #print(np.shape(np.array(pred)))
        #print(np.shape(np.array(pred)))
        pred_part = pred[:,:,j:j+5]
        #print(np.shape(np.array(pred)))
        index = np.argmax(pred_part, axis=2)
        #print(np.shape(np.array(index)))
        index = to_categorical(index, num_classes=5)
        #print(np.shape(np.array(index)))
        #print(index[i,:])
        pred_new += index
        #print(pred_new[0:3,:])

    pred_new = np.expand_dims(pred_new, axis=1)
    pred_new = np.argmax(pred_new, axis=2)
    max_voting_accuracies[k] = np.sum(np.equal(pred_new, y_test)) / y_test.shape[0]
    del max_voting_model
    del pred
    del pred_new
    print(k,"test_acc:", max_voting_accuracies[k])

In [ ]:
#np.savetxt('FR_max_voting_accuracies.txt', max_voting_accuracies, fmt='%.4f')

In [ ]:
# Average

average_accuracies = np.zeros((51,1))

for i in np.arange(2,50):
    n_models = i
    models_short = models_bag[0:n_models]
    average_model = average_ensemble(models_short, model_input)
    average_accuracies[i] = evaluate_test_accuracy(average_model)
    print(i,"test_acc:", average_accuracies[i])

In [ ]:
np.savetxt('FR_average_accuracies.txt', average_accuracies, fmt='%.4f')

#### 2.3.2 Non-bagging (evaluate simple combiners)

In [ ]:
# Max voting

models_short = models_nonbag[0:5]

pred_new = np.zeros((n_test,5))
max_voting_model = concatenate_ensemble(models_short, model_input)
pred = max_voting_model.predict(x_test, batch_size = 256)
pred = np.expand_dims(pred, axis=1)
for i in range(len(models_short)):
    j = i*5
    #print(i)
    #print(np.shape(np.array(pred)))
    #print(np.shape(np.array(pred)))
    pred_part = pred[:,:,j:j+5]
    #print(np.shape(np.array(pred)))
    index = np.argmax(pred_part, axis=2)
    #print(np.shape(np.array(index)))
    index = to_categorical(index, num_classes=5)
    #print(np.shape(np.array(index)))
    #print(index[i,:])
    pred_new += index
    #print(pred_new[0:3,:])

pred_new = np.expand_dims(pred_new, axis=1)
pred_new = np.argmax(pred_new, axis=2)
max_voting_accuracy = np.sum(np.equal(pred_new, y_test)) / y_test.shape[0]
print("max voting, test acc:", max_voting_accuracy)

In [ ]:
# Average

models_short = models_nonbag[0:5]

average_model = average_ensemble(models_short, model_input)
average_accuracy = evaluate_test_accuracy(average_model)
print("average, test acc:", average_accuracy)

### 2.4 Train and evaluate stacking combiners

#### 2.4.1 Bagging (train and evaluate stacking combiners)

In [ ]:
# Train ensemble once

n_models = 20

ensemble_model = small_stacking_ensemble(models_bag[0:n_models], model_input, n_models)
_ = compile_and_train(ensemble_model, num_epochs=5)
accuracy = evaluate_test_accuracy(ensemble_model)
print(n_models, "test_acc:", accuracy)

In [ ]:
# Train ensemble multiple times

basic_stacking_accuracies = np.zeros((51,1))

range_list = np.arange(0,50,5) 
range_list[0] = 2 # 2,5,10,15,20,...,45

for i in range_list:
    n_models = i
    ensemble_model = basic_stacking_ensemble(models_bag[0:n_models], model_input, i)
    _ = compile_and_train(ensemble_model, num_epochs=10)
    basic_stacking_accuracies[i] = evaluate_test_accuracy(ensemble_model)
    print(i,"test_acc:", basic_stacking_accuracies[i])


In [ ]:
np.savetxt('FR_basic_stacking_accuracies.txt', basic_stacking_accuracies, fmt='%.4f')

In [ ]:
# Evaluate ensemble

n_models = 15

ensemble_model = basic_stacking_ensemble(models_bag[0:n_models], model_input)
ensemble_model.load_weights('weights/FR/FR_basic_stacking_ensemble.01-0.99.hdf5')
print(evaluate_test_accuracy(ensemble_model))

 #### 2.4.2 Non-bagging (train and evaluate stacking combiners)

In [ ]:
# Train ensemble once and evaluate

models_short = models_nonbag[0:5]

ensemble_model = deep_stacking_ensemble(models_short, model_input, 0)
_ = compile_and_train(ensemble_model, num_epochs=20)
accuracy = evaluate_test_accuracy(ensemble_model)
print(len(models_short), "test_acc:", accuracy)

In [ ]:
# Evaluate ensemble

models_short = models_nonbag[0:5]

ensemble_model = deep_stacking_ensemble(models_short, model_input, 0)
ensemble_model.load_weights('weights/FR/FR_bn_deep_stacking_0_ensemble.08-0.69.hdf5')
print(evaluate_test_accuracy(ensemble_model))